## TIC collection import

Author: Melissa D

Last run: Jun 12, 2025

ISSUE LINK TODO

New threshold: threshold between 956_376 and 3_264_430

In [ ]:
# !pip install lsdb

In [2]:
import hats
import numpy as np
from dask.distributed import Client
from hats_import import CollectionArguments, pipeline_with_client, pipeline, VerificationArguments
from pathlib import Path
from hats_import.catalog.file_readers import CsvReader
from astropy.io import ascii
import lsdb

hats.__version__

'0.6'

In [3]:
## input paths:
raw_dir = Path("/data3/epyc/data3/hipscat/raw/gaia/")
file_list = list(raw_dir.glob("GaiaSource*"))
print("found", len(file_list), "files for import")

## Index division hints
global_min = 4295806720
global_max = 6917528997577384320
num_row_groups = 3933

increment = int((global_max - global_min) / num_row_groups)

divisions = np.append(np.arange(start=global_min, stop=global_max, step=increment), global_max)
divisions = divisions.tolist()

## Create a schema file, using the contents from the VOTable.

empty_astropy_table = ascii.read(file_list[0], format="ecsv", data_end=1)
empty_astropy_table.write("gaia_schema.parquet", overwrite=True)

found 3388 files for import


In [4]:
args = (
    CollectionArguments(
        completion_email_address="delucchi@andrew.cmu.edu",
        output_artifact_name="tic",
        output_path="/data3/epyc/data3/hats/catalogs/v06",
        progress_bar=True,
        simple_progress_bar=True,
    )
    .catalog(
        output_artifact_name="tic",
        input_file_list=file_list,
        file_reader=CsvReader(comment="#", schema_file="gaia_schema.parquet", compression="gzip"),
        ra_column="ra",
        dec_column="dec",
        sort_columns="source_id",
        use_schema_file="gaia_schema.parquet",
        highest_healpix_order=8,
        skymap_alt_orders=[2, 4, 6],
        pixel_threshold=2_000_000,
        row_group_kwargs={"num_rows": 200_000},
    )
    .add_margin(margin_threshold=10.0, is_default=True)
    .add_margin(margin_threshold=300.0, output_artifact_name="gaia_300arcs")
    .add_index(
        indexing_column="source_id",
        output_artifact_name="gaia_source_id_index",
        include_healpix_29=False,
        include_order_pixel=True,
        compute_partition_size=2_000_000_000,
        division_hints=divisions,
        drop_duplicates=False,
    )
)

In [ ]:
with Client(
    local_directory="/data3/epyc/data3/hats/tmp/",
    n_workers=20,
    threads_per_worker=1,
) as client:
    pipeline_with_client(args, client)

/astro/users/mmd11/.conda/envs/importenv/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34084 instead
  warnings.warn(
Mapping   :  93%|█████████▎| 3138/3388 [31:55<02:18,  1.80it/s]  2025-06-13 08:01:32,005 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:37507' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'map_to_pixels-b10c978e99d849b80843078f481aa6c7', 'map_to_pixels-95528c3bc60925baf382d0e288845bb6', 'map_to_pixels-559eab0767b220ef7f320fb3b5d4fe5c', 'map_to_pixels-4da9ba84875b1c9f282ba077fcdd7367', 'map_to_pixels-8089d9e8e17b5013ad6f130766c28ef9', 'map_to_pixels-dfb421981bef040cb5787f6509c58025', 'map_to_pixels-1bdaad8b962f8c4d32551c8b8f3c595c', 'map_to_pixels-947a6546aaea03fbb19fb0327a52e1f4', 'map_to_pixels-88466fa907df9a77a8ffe6d644ee7e41', 'map_to_pixels-15f899b07c4b79b660bd97dad37f570b', '

2025-06-13 08:01:32,070 - distributed.nanny - WARNING - Restarting worker
Reducing  :  11%|█▏        | 229/2016 [07:17<1:04:05,  2.15s/it]

In [ ]:
catalog = lsdb.open_catalog("/data3/epyc/data3/hats/catalogs/v06/gaia_dr3")
len(catalog.margin)

In [ ]:
catalog.plot_pixels()

In [ ]:
%% time
catalog.id_search(values={"source_id":35325045153309056}).compute()

In [ ]:
args = VerificationArguments(
    input_catalog_path="/data3/epyc/data3/hats/catalogs/v06/gaia_dr3/gaia",
    output_path="./verification/gaia_dr3",
)
pipeline(args)